In [1]:
import sys
sys.path.append('../src')
from db import *
from sqlalchemy import func
from sqlalchemy.orm import aliased
import numpy as np
from sqlalchemy import or_, and_

/Users/abuonomo/code/DataSquad/astro2020/venv/lib/python3.7/site-packages/tqdm/std.py:651: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
db_loc = '../scratch/test.sqlite'

In [3]:
engine = create_engine(f"sqlite:///{db_loc}")
Session = sessionmaker(bind=engine)
session = Session()

In [4]:
# session.query(PaperKeywords).join(Paper).filter(PaperKeywords.raw_keyword == "gamma").all()

In [5]:
year_counts = session.query(Paper.year, func.count(Paper.year)).group_by(Paper.year).all()

In [6]:
session.query(Keyword).filter(Keyword.keyword == "black hole").first().get_years(session)

[(1996, 5),
 (1997, 5),
 (1998, 4),
 (1999, 11),
 (2000, 5),
 (2001, 4),
 (2002, 3),
 (2003, 9),
 (2004, 9),
 (2005, 7),
 (2006, 10),
 (2007, 7),
 (2008, 7),
 (2009, 9),
 (2010, 11)]

In [7]:
session.query(Paper.year, func.count(Paper.year)) \
    .join(PaperKeywords) \
    .join(Keyword) \
    .filter(Keyword.keyword.ilike('mars')) \
    .group_by(Paper.year) \
    .all()

[(1996, 2),
 (1997, 1),
 (1998, 2),
 (1999, 1),
 (2001, 6),
 (2002, 3),
 (2004, 3),
 (2005, 6),
 (2006, 1),
 (2007, 1),
 (2009, 1),
 (2010, 3)]

In [8]:
session.query(Keyword).filter(Keyword.keyword.ilike('universe'))[0].papers[1].paper.get_feature_text()

"Electromagnetic counterparts of compact object mergers powered by the radioactive decay of r-process nuclei. The most promising astrophysical sources of kHz gravitational waves (GWs) are the inspiral and merger of binary neutron star(NS)/black hole systems. Maximizing the scientific return of a GW detection will require identifying a coincident electromagnetic (EM) counterpart. One of the most likely sources of isotropic EM emission from compact object mergers is a supernova-like transient powered by the radioactive decay of heavy elements synthesized in ejecta from the merger. We present the first calculations of the optical transients from compact object mergers that self-consistently determine the radioactive heating by means of a nuclear reaction network; using this heating rate, we model the light curve with a one-dimensional Monte Carlo radiation transfer calculation. For an ejecta mass ~10-2Msolar (10-3Msolar) the resulting light-curve peaks on a time-scale ~1d at a V-band lumi

In [9]:
paper_kwd = session.query(PaperKeywords).join(Keyword).filter(Keyword.keyword.ilike('black hole')).first()

In [10]:
# q = (
#     session.query(Paper).join(PaperKeywords).join(Keyword)
#     .filter(
#         or_(
#             Paper.title.contains(paper_kwd.raw_keyword), 
#             Paper.abstract.contains(paper_kwd.raw_keyword)
#         )
#     )
#     .filter(PaperKeywords != paper_kwd)
# )

In [11]:
# q.first().get_feature_text()

In [12]:
# session.query(PaperKeywords)[0]

In [13]:
# all_added = 0
# for paper_kwd in session.query(PaperKeywords)[0:10]:
#     added = 0
#     q = (
#         session.query(Paper).join(PaperKeywords)
#         .filter(
#             or_(
#                 Paper.title.contains(paper_kwd.raw_keyword), 
#                 Paper.abstract.contains(paper_kwd.raw_keyword)
#             )
#         )
#         .filter(PaperKeywords != paper_kwd)
#     )
#     papers = q.all()
#     print(len(papers))
#     for p in papers:
#         if paper_kwd.keyword not in [pk.keyword for pk in p.keywords]:
#             with session.no_autoflush:
#                 assoc = PaperKeywords(raw_keyword=paper_kwd.raw_keyword, score=np.nan)
#                 assoc.keyword = paper_kwd.keyword
#                 p.keywords.append(assoc)
#             added += 1
#     print(added)

In [14]:
paper_kwd = session.query(PaperKeywords)[0]
print(paper_kwd)

<PaperKeywords(paper_bibcode="2010AJ....140.1868W", keyword.keyword="sky survey")>


In [15]:
# session.query(PaperKeywords).join(Keyword).filter(Keyword.keyword == "black hole").all()

In [16]:
paper_kwd = session.query(PaperKeywords).join(Keyword).filter(Keyword.keyword == "black hole")[0]
print(paper_kwd)

<PaperKeywords(paper_bibcode="2010MNRAS.409..531R", keyword.keyword="black hole")>


In [52]:
print(paper_kwd.keyword_id)

935


In [115]:
q = (
    session.query(PaperKeywords)
    .join(Paper)
    .filter(PaperKeywords.keyword_id != paper_kwd.keyword_id)
    .filter(or_(Paper.title.contains(paper_kwd.raw_keyword), Paper.abstract.contains(paper_kwd.raw_keyword)))
)

In [116]:
q.count()

28884

In [46]:
bibs = [r.bibcode for r in q.all()]

In [47]:
# bibs

In [106]:
q2 = (
    session.query(PaperKeywords)
    .join(Paper)
#     .filter(PaperKeywords.keyword_id != paper_kwd.keyword_id)
    .filter(or_(Paper.abstract.contains(paper_kwd.raw_keyword), Paper.title.contains(paper_kwd.raw_keyword)))
)

In [107]:
print(q2)

SELECT paper_keywords.paper_bibcode AS paper_keywords_paper_bibcode, paper_keywords.keyword_id AS paper_keywords_keyword_id, paper_keywords.score AS paper_keywords_score, paper_keywords.raw_keyword AS paper_keywords_raw_keyword 
FROM paper_keywords JOIN papers ON papers.bibcode = paper_keywords.paper_bibcode 
WHERE (papers.abstract LIKE '%' || ? || '%') OR (papers.title LIKE '%' || ? || '%')


In [108]:
q2.count()

28990

In [96]:
len(q2.all())

647

In [49]:
bibs2 = [r.bibcode for r in q2.all()]

In [50]:
len(set(bibs2).intersection(set(bibs)))

106

In [51]:
len(bibs2)

106

In [29]:
session.rollback()

In [51]:
paper_kwd.keyword

<Keyword(keyword="black hole")>

In [116]:
session.query(Paper).join(PaperKeywords).join(Keyword).filter(Keyword.keyword != "black hole").count()

321083

In [105]:
q = (
    session.query(Paper).join(PaperKeywords).join(Keyword)
    .filter(
        or_(
            Paper.title.contains(paper_kwd.raw_keyword), 
            Paper.abstract.contains(paper_kwd.raw_keyword)
        )
    )
#     .filter(Keyword.id != paper_kwd.keyword.id)
)

In [107]:
len(q.all())

647

In [112]:
session.insert(Paper)

AttributeError: 'Session' object has no attribute 'insert'

In [120]:
session.query(Paper).join(PaperKeywords).join(Keyword)

In [6]:
# added = 0
# for paper_kwd in session.query(PaperKeywords)[0:10]:
#     print(paper_kwd)
#     q = (
#         session.query(Paper).join(PaperKeywords).join(Keyword)
#         .filter(
#             or_(
#                 Paper.title.contains(paper_kwd.raw_keyword), 
#                 Paper.abstract.contains(paper_kwd.raw_keyword)
#             )
#         )
#         .filter(Keyword.id != paper_kwd.keyword.id)
#     )
#     papers = q.all()
#     print(len(papers))
#     for p in papers:
# #         if paper_kwd.keyword not in [pk.keyword for pk in p.keywords]:
#         with session.no_autoflush:
#             assoc = PaperKeywords(raw_keyword=paper_kwd.raw_keyword, score=np.nan)
#             print()
# #             assoc.keyword = paper_kwd.keyword
# #             p.keywords.append(assoc)
#             import ipdb; ipdb.set_trace()
#         added += 1

In [14]:
added

573

In [5]:
session.rollback()
# session.close()

In [71]:
q.first()

('2010MNRAS.409..531R')

In [52]:
KeywordAlias = aliased(Keyword)
q = (
 session.query(Keyword, func.count(Keyword.id))
    .join(PaperKeywords)
    .join(Paper)
    .group_by(Keyword.id)
#     .filter(Paper.year == 2010)
    .order_by(func.count(Keyword.id).desc())
    .having(func.count() > 10)
    .having(funct.count() < )
)

In [50]:
print(q)

SELECT keywords.id AS keywords_id, keywords.keyword AS keywords_keyword, count(keywords.id) AS count_1 
FROM keywords JOIN paper_keywords ON keywords.id = paper_keywords.keyword_id JOIN papers ON papers.bibcode = paper_keywords.paper_bibcode GROUP BY keywords.id 
HAVING count(*) > ? ORDER BY count(keywords.id) DESC


In [51]:
q.count()

3225

In [11]:
session.query(Paper).count()

7245

In [37]:
# q[0:100]

In [12]:
tokens = ([pk.keyword.keyword for pk in paper.keywords] for paper in session.query(Paper).all())

In [14]:
nlp = get_spacy_nlp()

In [22]:
pm = PaperMiner(nlp)

In [23]:
pm.get_dictionary(session)

In [32]:
corpus = pm.get_corpus(session)

In [70]:
session.rollback()

In [71]:
session.close()